In [ ]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))
def parse_edge(s):
  user, follower = s.split("\t")
  return (int(user), int(follower))

def step(item):
  prev_v, prev_d, next_v = item[0], item[1][0], item[1][1]
  return (next_v, prev_d+[next_v])

def complete(item):
  v, old_d, new_d = item[0], item[1][0], item[1][1]
  return (v, old_d if old_d is not None else new_d)

 # number of partitions
n = 4
edges = sc.textFile("/data/twitter/twitter_sample.txt").map(parse_edge).cache()
forward_edges = edges.map(lambda e: (e[1], e[0])).partitionBy(n).persist()
x = 12
d = 0
y=34
distances = sc.parallelize([(x,[x])]).partitionBy(n)
while True:
    #Get RDD's with neighbour Give output with edges and array of list
    
    new_distances = distances.join(forward_edges, n).map(step).persist()
    #new_distances = distances.fullOuterJoin(candidates, n).map(complete, True).persist()
    #print(new_distances.take(5))
    #break
    count = new_distances.filter(lambda x:x[0] == y).count()
    if count == 0:
        d+= 1
        distances = new_distances
        #print("Distances RDD count",distances.count(),"d = ", d)
    else:
        break
path=new_distances.filter(lambda x:x[0] == y).collect()
print(",".join(map(str,path[0][1])))
sc.stop()